# Cleaning Dataset 3 'YOJ' Column with KNN

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## EXPLORE THE DATASET

In [4]:
# Load the dataset
df = pd.read_csv('dataset3.csv')
df.head()

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,...,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,...,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,...,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,...,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,...,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [5]:
# Check for missing values in the 'CAR_AGE' column
missing_values = df['YOJ'].isna().sum()
print(f"Missing values in 'YOJ': {missing_values}")

Missing values in 'YOJ': 548


## CLEAN THE DATASET

In [7]:
# Function to clean and convert columns to numeric
def clean_numeric_column(df, column):
    df[column] = df[column].replace(r'[\$,]', '', regex=True).astype(float)

# Columns to convert
columns_to_convert = ['INCOME', 'HOME_VAL', 'BLUEBOOK', 'OLDCLAIM', 'CLM_AMT']

for column in columns_to_convert:
    clean_numeric_column(df, column)

# After cleaning, confirm that data types are correct
print(df.dtypes)

ID              int64
KIDSDRIV        int64
BIRTH          object
AGE           float64
HOMEKIDS        int64
YOJ           float64
INCOME        float64
PARENT1        object
HOME_VAL      float64
MSTATUS        object
GENDER         object
EDUCATION      object
OCCUPATION     object
TRAVTIME        int64
CAR_USE        object
BLUEBOOK      float64
TIF             int64
CAR_TYPE       object
RED_CAR        object
OLDCLAIM      float64
CLM_FREQ        int64
REVOKED        object
MVR_PTS         int64
CLM_AMT       float64
CAR_AGE       float64
CLAIM_FLAG      int64
URBANICITY     object
dtype: object


In [8]:
# Split into numerical and categorical columns
numerical_columns = ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'MVR_PTS', 'INCOME', 'HOME_VAL', 'BLUEBOOK', 'CLM_AMT']
categorical_columns = ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']

# Ensure the lists are correctly defined
print("Numerical columns:", numerical_columns)
print("Categorical columns:", categorical_columns)

Numerical columns: ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'MVR_PTS', 'INCOME', 'HOME_VAL', 'BLUEBOOK', 'CLM_AMT']
Categorical columns: ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']


## CHECK FOR CORRELATIONS IN THE DATASET

In [10]:
correlation_matrix = df[numerical_columns].corr()

# Print correlation values for YOJ
print(correlation_matrix['YOJ'].sort_values(ascending=False))

YOJ         1.000000
INCOME      0.293899
HOME_VAL    0.272198
BLUEBOOK    0.148086
AGE         0.140878
HOMEKIDS    0.077707
KIDSDRIV    0.042971
TIF         0.023921
TRAVTIME   -0.017152
CLM_AMT    -0.026799
MVR_PTS    -0.041593
Name: YOJ, dtype: float64


In [11]:
print(df.groupby('PARENT1')['YOJ'].mean())

print(df.groupby('MSTATUS')['YOJ'].mean())

print(df.groupby('GENDER')['YOJ'].mean())

print(df.groupby('EDUCATION')['YOJ'].mean())

print(df.groupby('CAR_USE')['YOJ'].mean())

print(df.groupby('CAR_TYPE')['YOJ'].mean())

print(df.groupby('RED_CAR')['YOJ'].mean())

print(df.groupby('REVOKED')['YOJ'].mean())

print(df.groupby('URBANICITY')['YOJ'].mean())

print(df.groupby('OCCUPATION')['YOJ'].mean())

PARENT1
No     10.536421
Yes    10.057480
Name: YOJ, dtype: float64
MSTATUS
Yes     10.947549
z_No     9.769388
Name: YOJ, dtype: float64
GENDER
M      10.826097
z_F    10.170802
Name: YOJ, dtype: float64
EDUCATION
<High School     10.060924
Bachelors        10.571482
Masters          11.083802
PhD              10.904387
z_High School    10.032955
Name: YOJ, dtype: float64
CAR_USE
Commercial    10.613523
Private       10.392695
Name: YOJ, dtype: float64
CAR_TYPE
Minivan        10.808101
Panel Truck    11.102978
Pickup         10.504728
Sports Car      9.755814
Van            11.002304
z_SUV          10.083975
Name: YOJ, dtype: float64
RED_CAR
no     10.337713
yes    10.807992
Name: YOJ, dtype: float64
REVOKED
No     10.489364
Yes    10.364775
Name: YOJ, dtype: float64
URBANICITY
Highly Urban/ Urban      10.636364
z_Highly Rural/ Rural     9.826598
Name: YOJ, dtype: float64
OCCUPATION
Clerical         11.446053
Doctor           11.332248
Home Maker        5.827715
Lawyer           11.46

## PREPARE THE DATASET FOR TRAINING 

In [13]:
#Select the relevant numerical and categorical features for imputation
features_for_imputation = ['YOJ', 'INCOME', 'HOME_VAL', 'BLUEBOOK', 'AGE', 'PARENT1', 
                           'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 'CAR_TYPE', 
                           'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']

# Copy the relevant columns into a new DataFrame
df_impute = df[features_for_imputation].copy()

In [14]:
# Encode categorical variables
label_encoders = {}
for col in ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 
            'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']:
    le = LabelEncoder()
    df_impute[col] = le.fit_transform(df_impute[col].astype(str))
    label_encoders[col] = le

## TRAIN THE DATASET

In [16]:
# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)

In [17]:
# Apply KNN imputation to the data
df_imputed = knn_imputer.fit_transform(df_impute)

In [18]:
# Replace the original YOJ column in the original DataFrame with the imputed values
df['YOJ'] = df_imputed[:, 0]

In [19]:
# Check if missing values are imputed
print(df['YOJ'].isnull().sum())

0


In [20]:
# Save the imputed dataset to a CSV file
df.to_csv('KNNimputed_ALL_dataset3.csv', index=False)

## ACCURACY

In [22]:
# Create a copy of the original data and introduce random missing values
df_test = df.copy()

# Define the fraction of data to randomly remove for testing (e.g., 20%)
missing_fraction = 0.2
np.random.seed(42)  # For reproducibility

# Randomly select 20% of the non-missing YOJ values to be set as NaN for testing
non_null_indices = df_test[df_test['YOJ'].notnull()].index
test_indices = np.random.choice(non_null_indices, size=int(missing_fraction * len(non_null_indices)), replace=False)
original_values = df_test.loc[test_indices, 'YOJ']  # Store the original values for comparison
df_test.loc[test_indices, 'YOJ'] = np.nan  # Introduce missing values

In [23]:
# Impute the missing values using KNN (reusing the previous code)
# Make sure you use the same feature selection and encoding as before
df_impute_test = df_test[features_for_imputation].copy()

# Apply the label encoding for categorical features
for col in ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'CAR_USE', 
            'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY', 'OCCUPATION']:
    le = label_encoders[col]
    df_impute_test[col] = le.transform(df_impute_test[col].astype(str))

# Apply KNN imputer to the test data
df_imputed_test = knn_imputer.fit_transform(df_impute_test)

In [24]:
# Compare the imputed values with the original values
# Get the imputed YOJ values
imputed_values = df_imputed_test[test_indices, 0]

In [25]:
# Calculate the mean squared error (or any other metric) between original and imputed values
mse = mean_squared_error(original_values, imputed_values)
print(f'Mean Squared Error for YOJ imputation: {mse}')

# Mean Absolute Error (MAE)
mae = mean_absolute_error(original_values, imputed_values)
print(f'Mean Absolute Error for YOJ imputation: {mae}')

# R-squared (R²)
r2 = r2_score(original_values, imputed_values)
print(f'R-squared (R²): {r2}')

Mean Squared Error for YOJ imputation: 11.745275339805826
Mean Absolute Error for YOJ imputation: 2.598252427184466
R-squared (R²): 0.26293669073644266


MSE of 11.745 suggests that, on average, the squared differences between the imputed and actual values are relatively moderate.                  
MAE of 2.598 suggests that, on average, the imputed values are about 2.6 years off from the actual values.                     
R² of 0.263 indicates that about 26.3% of the variance in YOJ is explained by the imputed values. This is relatively low, suggesting that the imputation model explains a **modest** portion of the variability in YOJ